In [14]:
import numpy as np

def onehot_chd_pitch_shift(onehot, shift):
    ret = np.zeros_like(onehot)
    for i in range(3):
        ret[:, 12*i:12*i+12] = np.roll(onehot[:, 12*i:12*i+12], shift=shift, axis=-1)
    return ret


In [15]:
from data.utils import midi_to_one_hot_chd
onehot_chord = midi_to_one_hot_chd(midi_fpath="test_data/seg_0.mid",chd_fpath="test.out")

In [16]:
print(onehot_chord)
print(onehot_chord.shape)

[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(32, 36)


In [17]:
from data.utils import chd_to_midi_file

chd_to_midi_file(onehot_chord, "test_data/seg_0_chord.mid")

In [18]:
for i in range(-6,6):
    shift_i = onehot_chd_pitch_shift(onehot_chord, i)
    chd_to_midi_file(shift_i, f"test_data/seg_0_chord_shift_{i}.mid")


In [19]:
from data.utils import pr_mat_pitch_shift
# 0 = Bass
# 1 = Guitar
# 2 = Piano
# 3 = String

import muspy
music = muspy.read_midi("test_data/seg_0.mid")

multi_prmat_2c = np.zeros((4,2,128,128))

for track in music.tracks:
    if track.is_drum:
        continue
    if track.program == 32:
        track_idx = 0
    if track.program == 24:
        track_idx = 1
    if track.program == 0:
        track_idx = 2
    if track.program == 48:
        track_idx = 3

    for note in track.notes:
        start_time = note.start
        duration = note.duration
        pitch = note.pitch
        multi_prmat_2c[track_idx, 0, start_time, pitch] = 1.0
        for d in range(1, duration):
            if start_time + d < 128:
                multi_prmat_2c[track_idx, 1, start_time + d, pitch] = 1.0

In [20]:
from data.utils import prmat2c_to_midi_file
for i in range(-6,6):
    shift_prmat = pr_mat_pitch_shift(multi_prmat_2c, i)
    reduction = np.sum(shift_prmat, axis=0)
    prmat2c_to_midi_file(reduction, f"test_data/seg_0_shift_{i}.mid")